In [52]:
import pandas as pd
import sklearn as sk
import cPickle as p
import dill
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model

#define load the data loading method
def load_json_yelp(filepath):
    with open(filepath, 'rb') as f:
        data = f.readlines()
        data = map(lambda x: x.rstrip(), data)
        data_json_str = "[" + ','.join(data) + "]"
        data_df = pd.read_json(data_json_str)
    return data_df

#the place for class definition
class  Estimator(sk.base.BaseEstimator, sk.base.RegressorMixin):
    def __init__(self,model):
        self.est = model
        pass
    def fit(self,X,y):
        test_param = {'alpha':[0.01,0.05,0.1,0.3,0.5,1,10]}
        gscv = GridSearchCV(self.est, test_param)
        gscv.fit(X,y)
        self.est = gscv.best_estimator_
        #self.est.fit(X,y)
        return self
    
    def score(self,X,y):
        return self.est.score(X_test,y)
    
    def predict(self, X):
        return self.est.predict(X)

#here is the place for trnasformer
class Transformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    def __init__(self):
        self.vectorizer = DictVectorizer()
        
    def fit(self,X,y=None):
        X = X['categories']
        new_X = list()
        for x in X:
            dic = dict()
            for nx in x:
                dic[nx]=1
            dic['pz']=1
            new_X.append(dic)
        self.vectorizer.fit_transform(new_X)
        return self
    
    def transform(self,X):
        X = X['categories']
        new_X = list()
        if type(X) is list: #for test data
            dic = dict()
            for x in X:
                dic[x]=1
            dic['pz']=1
            new_X.append(dic)
        else:
            for x in X:
                dic = dict()
                for nx in x:
                    dic[nx]=1
                dic['pz']=1
                new_X.append(dic)
        X_trans = self.vectorizer.transform(new_X)
        return X_trans


#data loading
filepath = 'yelp_train.json'
yelp_data = load_json_yelp(filepath)

#use the transformer
X = yelp_data
y = yelp_data['stars']
#the place for generating cross validation group
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#tuned_variable = {'estimate__n_neighbors':[5,7,9,11,13,15]}
pipeline = Pipeline([('getcols',Transformer()),('estimate',Estimator(linear_model.Ridge()))])
print 'done'

done


In [53]:
presult = pipeline.fit(X,y)

with open('categories_model','wb') as output:
    dill.dump(presult,output)

with open("categories_model", 'rb') as in_strm:
    datastruct = dill.load(in_strm)

print datastruct

presult = datastruct.predict(X)

print presult
print type(presult)

print 'done'


Pipeline(steps=[('getcols', Transformer()), ('estimate', Estimator(model=None))])
[ 3.38184949  3.37916397  3.29936325 ...,  4.43342136  3.61306141
  3.9587923 ]
<type 'numpy.ndarray'>
done
